# Train the Tokenizer

In [19]:
from tokenizers import BertWordPieceTokenizer, ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing
from pathlib import Path
import os

# paths = [str(x) for x in Path(".").glob("text_split/*.txt")]
paths = ["/home/michael/Workspace/NLP-Chess/dataset_3_no_sep_unique.txt"]

tokenizer_folder = os.path.realpath("./roberta-harmon")

In [ ]:




# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer(lowercase=False)

# Customize training
tokenizer.train(files=paths, vocab_size=16384, min_frequency=1,
                show_progress=True,
                special_tokens=[                                "<s>",
                                "<pad>",
                                "</s>",
                                "<unk>",
                                "<mask>", "<MOVE_SEP>"])
#Save the Tokenizer to disk
tokenizer.save_model(tokenizer_folder)

In [3]:
tokenizer.save_model(tokenizer_folder)

['/home/michael/Workspace/transformers/examples/pytorch/language-modeling/bert-harmon/vocab.txt']

In [6]:
import os

bert = True
if bert != True:
    # Create the tokenizer using vocab.json and mrege.txt files
    tokenizer = ByteLevelBPETokenizer(
        os.path.abspath(os.path.join(tokenizer_folder,'vocab.json')),
        os.path.abspath(os.path.join(tokenizer_folder,'merges.txt'))
    )
else:
    tokenizer = BertWordPieceTokenizer(os.path.abspath(os.path.join(tokenizer_folder, 'vocab.txt')))
# Prepare the tokenizer
# tokenizer._tokenizer.post_processor = BertProcessing(
#     ("</s>", tokenizer.token_to_id("</s>")),
#     ("<s>", tokenizer.token_to_id("<s>")),
# )
tokenizer.enable_truncation(max_length=128)
# Test the tokenizer
tokenizer.encode("rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1 White <MOVE_SEP> e2e4")
# Show the tokens created
tokenizer.encode("rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1 White <MOVE_SEP> e2e4").tokens

['[CLS]',
 'rnbqkbnr',
 '/',
 'pppppppp',
 '/',
 '8',
 '/',
 '8',
 '/',
 '8',
 '/',
 '8',
 '/',
 'pppppppp',
 '/',
 'rnbqkbnr',
 'w',
 'kqkq',
 '-',
 '0',
 '1',
 'white',
 '<',
 'move',
 '_',
 'sep',
 '>',
 'e2e4',
 '[SEP]']

In [4]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=16384,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config=config)
print('Num parameters: ',model.num_parameters())

Num parameters:  56115712


In [5]:
MAX_LEN = 128

from transformers import RobertaTokenizer
# Create the tokenizer from a trained one
tokenizer = RobertaTokenizer.from_pretrained(tokenizer_folder, max_len=MAX_LEN)

print(tokenizer)

file /home/michael/Workspace/transformers/examples/pytorch/language-modeling/roberta-harmon/config.json not found


PreTrainedTokenizer(name_or_path='/home/michael/Workspace/transformers/examples/pytorch/language-modeling/roberta-harmon', vocab_size=16384, model_max_len=128, is_fast=False, padding_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'sep_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'cls_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True)})


In [2]:
from transformers import BertConfig
from transformers import BertForMaskedLM

# Set a configuration for our RoBERTa model
config = BertConfig(
    vocab_size=8192,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)
# Initialize the model from a configuration without pretrained weights
model = BertForMaskedLM(config=config)
print('Num parameters: ',model.num_parameters())

Num parameters:  49816064


In [3]:

from transformers import BertTokenizer
MAX_LEN = 128
# Create the tokenizer from a trained one
tokenizer = BertTokenizer.from_pretrained(tokenizer_folder + "/vocab.txt", max_len=MAX_LEN)



/home/michael/anaconda3/envs/hugging-face/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1642: FutureWarning: Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  warnings.warn(


In [9]:
from transformers import DistilBertTokenizer
from transformers.file_utils import is_remote_url

print(os.path.isfile(tokenizer_folder))
print(is_remote_url(tokenizer_folder))

pretrained_model_name_or_path = str(tokenizer_folder)

print(os.path.isfile(pretrained_model_name_or_path))
print(is_remote_url(pretrained_model_name_or_path))

MAX_LEN = 128
# Create the tokenizer from a trained one
print(tokenizer_folder)
tokenizer = DistilBertTokenizer.from_pretrained([os.path.abspath(os.path.join(tokenizer_folder,'vocab.json')),
    os.path.abspath(os.path.join(tokenizer_folder,'merges.txt'))])

False
False
False
False
/home/michael/Workspace/transformers/examples/pytorch/language-modeling/bert-harmon


HTTPError: 404 Client Error: Not Found for url: https://huggingface.co/api/models/%5B'/home/michael/Workspace/transformers/examples/pytorch/language-modeling/bert-harmon/vocab.json',%20'/home/michael/Workspace/transformers/examples/pytorch/language-modeling/bert-harmon/merges.txt'%5D

# Build the Dataset

In [6]:
from datasets import load_dataset
from datasets import set_caching_enabled
raw_datasets = load_dataset('text', data_files="/home/michael/Workspace/NLP-Chess/src/data/dataset_3_sep.txt",
                            split='train')

#cut size in half
raw_datasets = raw_datasets.shuffle(seed=42).select(range(int(len(raw_datasets) / 2.5)))

#raw_datasets = raw_datasets.select(range(10000))
raw_datasets = raw_datasets.train_test_split()


def tokenize_function(examples):
    return tokenizer(examples["text"], padding='max_length', truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True, keep_in_memory=True, num_proc=20, remove_columns=["text"])

Using custom data configuration default-c7a673a1a118bcc9
Reusing dataset text (/home/michael/.cache/huggingface/datasets/text/default-c7a673a1a118bcc9/0.0.0/e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5)
Loading cached shuffled indices for dataset at /home/michael/.cache/huggingface/datasets/text/default-c7a673a1a118bcc9/0.0.0/e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5/cache-83f8635946c56403.arrow


In [7]:
# tokenized_datasets = raw_datasets
#     with open("dataset-tokenized.obj", 'wb') as f:
#         pickle.dump(tokenized_datasets, f)

# block_size = tokenizer.model_max_length

block_size = 128

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    #result["labels"] = result["input_ids"].copy()
    return result
print(tokenized_datasets["train"][1])
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=20,
    keep_in_memory=True
)

print("grouping complete")

small_train_dataset = lm_datasets["train"].shuffle(seed=42).select(range(1000))
# small_train_dataset["labels"] = small_train_dataset["input_ids"]
small_eval_dataset = lm_datasets["test"].shuffle(seed=42).select(range(1000))
# small_eval_dataset["labels"] = small_eval_dataset["input_ids"]
full_train_dataset = lm_datasets["train"]
full_eval_dataset = lm_datasets["test"]

{'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'input_ids': [0, 348, 20, 369, 20, 29, 20, 29, 20, 24, 53, 25, 20, 29, 20, 53, 53, 53, 22, 53, 53, 53, 53, 20, 55, 51, 39, 54, 48, 39, 51, 55, 264, 226, 48, 54, 468, 263, 281, 280, 226, 39, 277, 80, 270, 50, 52, 59, 42, 68, 56, 42, 53, 35, 295, 28, 73, 26, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
grouping complete


In [8]:
len(tokenized_datasets["train"])

4157124

In [11]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    per_device_train_batch_size=392,
    output_dir='./output-robert', 
    num_train_epochs=100,)

from transformers import Trainer, DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm_probability=0.15,
)

trainer = Trainer(
    model=model, args=training_args, train_dataset=full_train_dataset, eval_dataset=full_eval_dataset, data_collator=data_collator
)

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running training *****
  Num examples = 4157124
  Num Epochs = 100
  Instantaneous batch size per device = 392
  Total train batch size (w. parallel, distributed & accumulation) = 392
  Gradient Accumulation steps = 1
  Total optimization steps = 1060500


Step,Training Loss
500,1.602000
1000,1.016400
1500,0.819600
2000,0.723900
2500,0.664300
3000,0.620600
3500,0.591800
4000,0.568000
4500,0.550100
5000,0.535100


Saving model checkpoint to ./output-robert/checkpoint-500
Configuration saved in ./output-robert/checkpoint-500/config.json
Model weights saved in ./output-robert/checkpoint-500/pytorch_model.bin
Saving model checkpoint to ./output-robert/checkpoint-1000
Configuration saved in ./output-robert/checkpoint-1000/config.json
Model weights saved in ./output-robert/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to ./output-robert/checkpoint-1500
Configuration saved in ./output-robert/checkpoint-1500/config.json
Model weights saved in ./output-robert/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to ./output-robert/checkpoint-2000
Configuration saved in ./output-robert/checkpoint-2000/config.json
Model weights saved in ./output-robert/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to ./output-robert/checkpoint-2500
Configuration saved in ./output-robert/checkpoint-2500/config.json
Model weights saved in ./output-robert/checkpoint-2500/pytorch_model.bin
Saving m

KeyboardInterrupt: 

In [15]:
!pwd

/home/michael/Workspace/transformers/examples/pytorch/language-modeling


In [9]:
len(raw_datasets)

13857084